## OR - Assignment 1

##### Author: Joaquim Marset Alsina

### Imports

In [1]:
import os
import random
from generator import ImageGenerator, TrainImageGenerator, TrainBalancedImageGenerator
from train import Trainer
from load_utils import *
from segmentation_utils import extract_train_segmentations, sort_objects_to_balance
from model_utils import mobile_name
from other_utils import *

### Constants

In [2]:
root_path = "./"
train_val_path = os.path.join(root_path, 'VOCtrainval_06-Nov-2007', 'VOCdevkit', 'VOC2007')
seg_objects_path = os.path.join(train_val_path, "SegmentationObject")
seg_classes_path = os.path.join(train_val_path, "SegmentationClass")
annotations_path = os.path.join(train_val_path, "Annotations")
images_path = os.path.join(train_val_path, "JPEGImages")

train_split_path = os.path.join(root_path, 'voc_train.txt')
val_split_path = os.path.join(root_path, 'voc_val.txt')

segmentations_pickle_path = os.path.join(root_path, "segmentations.pkl")
train_classes_counts_path = os.path.join(root_path, "trian_classes_counts.pkl")

results_path = os.path.join(root_path, 'results')
results_mobile_path = os.path.join(results_path, mobile_name)

models_path = os.path.join(root_path, 'models')
models_mobile_path = os.path.join(models_path, mobile_name)

In [3]:
seed = 1412
image_size = 224
val_percentage = 0.2
prob_augment = 0.5
batch_size = 32
num_epochs = 50
num_classes = get_num_classes()

### Create required folders

In [4]:
if not exists_path(results_path):
    create_results_folder(results_path)
    create_results_folder(results_mobile_path)

In [5]:
if not exists_path(models_path):
    create_results_folder(models_path)
    create_results_folder(models_mobile_path)

### Load train and validation split

In [6]:
train_names, val_names = read_train_val_split(train_split_path, val_split_path)

random.Random(seed).shuffle(train_names)
random.Random(seed).shuffle(val_names)

### Extract segmentation objects

In [7]:
if not exists_path(segmentations_pickle_path):
    train_segmentations = extract_train_segmentations(images_path, seg_objects_path, seg_classes_path, train_names, image_size)
    save_dict_to_pickle(segmentations_pickle_path, train_segmentations)
else:
    train_segmentations = load_pickle_dict(segmentations_pickle_path)

### Experiments

In [8]:
# Unchanged during all experiments
val_generator = ImageGenerator(batch_size, images_path, annotations_path, val_names, image_size, seed)

In [9]:
fine_tune_epochs = 30

def run_experiment_fine_tune(model_name, results_path, models_path, train_gen, experiment_name, model_file, verbose=0):
    file_name, plot_title = experiment_name
    trainer = Trainer(results_path, models_path, model_name, image_size, num_classes)
    trainer.train_fine_tune(num_epochs, fine_tune_epochs, train_gen, val_generator, file_name, plot_title, model_file, verbose)

#### MobileNetv2 - No augmentation

In [ ]:
augmentation_mode = AugmentationMode.NoAugmentation

train_generator = ImageGenerator(batch_size, images_path, annotations_path, train_names, image_size, seed)

experiment_name_file = 'no_augmentation'
experiment_name_title = 'No augmentation'
model_file = experiment_name_file

run_experiment_fine_tune(mobile_name, results_mobile_path, models_mobile_path, 
    train_generator, (experiment_name_file, experiment_name_title), model_file)

#### MobileNetv2 - Augmentation without overlap

In [ ]:
augmentation_mode = AugmentationMode.Augmentation

for num_to_place in [1, 3, 5]:

    train_generator = TrainImageGenerator(batch_size, images_path, annotations_path, train_names, 
    image_size, train_segmentations, augmentation_mode, num_to_place, prob_augment, seed)

    experiment_name_file = f'augmentation_{num_to_place}'
    experiment_name_title = f'Augmentation - Placing {num_to_place}'
    model_file = experiment_name_file

    run_experiment_fine_tune(mobile_name, results_mobile_path, models_mobile_path, 
        train_generator, (experiment_name_file, experiment_name_title), model_file)

    print(f'Finished experiment placing {num_to_place} objects')

#### MobileNetv2 - Augmentation without overlap applying random transformations

In [ ]:
augmentation_mode = AugmentationMode.AugmentationTransform

for num_to_place in [1, 3, 5]:

    train_generator = TrainImageGenerator(batch_size, images_path, annotations_path, train_names, 
        image_size, train_segmentations, augmentation_mode, num_to_place, prob_augment, seed)

    experiment_name_file = f'augmentation_transform_{num_to_place}'
    experiment_name_title = f'Augmentation - Random transformations - Placing {num_to_place}'
    model_file = experiment_name_file

    run_experiment_fine_tune(mobile_name, results_mobile_path, models_mobile_path, 
        train_generator, (experiment_name_file, experiment_name_title), model_file)

    print(f'Finished experiment placing {num_to_place} objects')

#### MobileNetv2 - Augmentation without overlap keeping the same class proportion

In [25]:
if not exists_path(train_classes_counts_path):
    train_classes_counts = extract_train_classes_counts(annotations_path, train_names)
    save_dict_to_pickle(train_classes_counts_path, train_classes_counts)
else:
    train_classes_counts = load_pickle_dict(train_classes_counts_path)

In [ ]:
augmentation_mode = AugmentationMode.AugmentationSameProportion

for fraction in [4, 3, 2]:

    place_per_label, objects_per_label = sort_objects_to_balance(train_classes_counts, train_segmentations, fraction)

    experiment_name_file = f'augmentation_proportions_{fraction}'
    experiment_name_title = f'Augmentation - Same class proportion 1/{fraction} of person'
    model_file = experiment_name_file

    histogram_path = os.path.join(results_path, f'{experiment_name_file}_{fraction}_fine_tune_class_histogram.png')

    train_generator = TrainBalancedImageGenerator(batch_size, images_path, annotations_path, train_names, 
        image_size, place_per_label, objects_per_label, augmentation_mode, histogram_path, seed)

    run_experiment_fine_tune(mobile_name, results_mobile_path, models_mobile_path, 
        train_generator, (experiment_name_file, experiment_name_title), model_file)

    print(f'Finished experiment balancing up to 1/{fraction} of person objects')

#### MobileNetv2 - Augmentation without overlap applying random transformations and keeping the same class proportion

In [ ]:
augmentation_mode = AugmentationMode.AugmentationTransformSameProportion

for fraction in [4, 3, 2]:

    place_per_label, objects_per_label = sort_objects_to_balance(train_classes_counts, train_segmentations, fraction)

    experiment_name_file = f'augmentation_transform_proportions_{fraction}'
    experiment_name_title = f'Augmentation - Random transformations - Same class proportion 1/{fraction} of person'
    model_file = experiment_name_file

    histogram_path = os.path.join(results_path, f'{experiment_name_file}_{fraction}_fine_tune_class_histogram.png')

    train_generator = TrainBalancedImageGenerator(batch_size, images_path, annotations_path, train_names, 
        image_size, place_per_label, objects_per_label, augmentation_mode, histogram_path, seed)

    run_experiment_fine_tune(mobile_name, results_mobile_path, models_mobile_path, 
        train_generator, (experiment_name_file, experiment_name_title), model_file)

    print(f'Finished experiment balancing up to 1/{fraction} of person objects')

#### MobileNetv2 - Augmentation with overlap

In [ ]:
augmentation_mode = AugmentationMode.AugmentationOverlap

for num_to_place in [1, 3, 5]:

    train_generator = TrainImageGenerator(batch_size, images_path, annotations_path, train_names, 
        image_size, train_segmentations, augmentation_mode, num_to_place, prob_augment, seed)

    experiment_name_file = f'augmentation_overlap_{num_to_place}'
    experiment_name_title = f'Augmentation - Overlap - Placing {num_to_place}'
    model_file = experiment_name_file

    run_experiment_fine_tune(mobile_name, results_mobile_path, models_mobile_path, 
        train_generator, (experiment_name_file, experiment_name_title), model_file)

    print(f'Finished experiment placing {num_to_place} objects')

#### MobileNetv2 - Augmentation with overlap and random transformations

In [ ]:
augmentation_mode = AugmentationMode.AugmentationOverlapTransform

for num_to_place in [1, 3, 5]:

    train_generator = TrainImageGenerator(batch_size, images_path, annotations_path, train_names, 
        image_size, train_segmentations, augmentation_mode, num_to_place, prob_augment, seed)

    experiment_name_file = f'augmentation_transform_overlap_{num_to_place}'
    experiment_name_title = f'Augmentation - Overlap - Random transformations - Placing {num_to_place}'
    model_file = experiment_name_file

    run_experiment_fine_tune(mobile_name, results_mobile_path, models_mobile_path, 
        train_generator, (experiment_name_file, experiment_name_title), model_file)

    print(f'Finished experiment placing {num_to_place} objects')

#### MobileNetv2 - Augmentation with overlap and equal proportions of labels

In [ ]:
augmentation_mode = AugmentationMode.AugmentationOverlapSameProportion

for fraction in [4, 3, 2]:

    place_per_label, objects_per_label = sort_objects_to_balance(train_classes_counts, train_segmentations, fraction)

    experiment_name_file = f'augmentation_overlap_proportions_{fraction}'
    experiment_name_title = f'Augmentation - Overlap - Same class proportion 1/{fraction} of person'
    model_file = experiment_name_file

    histogram_path = os.path.join(results_path, f'{experiment_name_file}_{fraction}_fine_tune_class_histogram.png')

    train_generator = TrainBalancedImageGenerator(batch_size, images_path, annotations_path, train_names, 
        image_size, place_per_label, objects_per_label, augmentation_mode, histogram_path, seed)

    run_experiment_fine_tune(mobile_name, results_mobile_path, models_mobile_path, 
        train_generator, (experiment_name_file, experiment_name_title), model_file)

    print(f'Finished experiment balancing up to 1/{fraction} of person objects')

#### MobileNetv2 - Augmentation with overlap applying random transformations and keeping the same class proportion

In [ ]:
augmentation_mode = AugmentationMode.AugmentationOverlapTransformSameProportion

for fraction in [4, 3, 2]:

    place_per_label, objects_per_label = sort_objects_to_balance(train_classes_counts, train_segmentations, fraction)

    experiment_name_file = f'augmentation_overlap_transform_proportions_{fraction}'
    experiment_name_title = f'Augmentation - Overlap - Random transformations - Same class proportion 1/{fraction} of person'
    model_file = experiment_name_file

    histogram_path = os.path.join(results_path, f'{experiment_name_file}_{fraction}_fine_tune_class_histogram.png')

    train_generator = TrainBalancedImageGenerator(batch_size, images_path, annotations_path, train_names, 
        image_size, place_per_label, objects_per_label, augmentation_mode, histogram_path, seed)

    run_experiment_fine_tune(mobile_name, results_mobile_path, models_mobile_path, 
        train_generator, (experiment_name_file, experiment_name_title), model_file)

    print(f'Finished experiment balancing up to 1/{fraction} of person objects')